# Future Drought Targets

The purpose of this notebook is to derive the future drought targets for "calibrating" a custom weather generator.

## Imports and Parameters

In [1]:
%matplotlib inline

In [2]:
import os
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import stats as sstats
from math import exp, log, pow
import datetime as dt
import seaborn as sns
import pickle

In [3]:
plt.rcParams['svg.fonttype'] = 'none'

In [4]:
IN_DIR1 = r'C:\Users\nmartin\Documents\EAA_HCP\Data\SwRI_Processed\Processed_Historical\SPEI'
IN_DIR2 = r'C:\Users\nmartin\Documents\EAA_HCP\Data\SwRI_Processed\DayMet_GridIntersect'
OUT_DIR = r'C:\Users\nmartin\Documents\EAA_HCP\Data\SwRI_Processed\Processed_Historical\SPEI'

In [5]:
# full basin intersection dictionary
InFiler = os.path.normpath( os.path.join( IN_DIR2, "BasWeightsGDF.pkl" ) )
with open( InFiler, 'rb' ) as IF:
    BasWeightsDF = pickle.load( IF )
# end with
BAS_KEYS = sorted( BasWeightsDF.keys() )

In [6]:
ExtractTS = pd.Timestamp( 2022, 7, 1, 0 )

In [7]:
MonDict = { 1 : "Jan", 2 : "Feb", 3 : "Mar", 4 : "Apr", 5 : "May", 6 : "Jun", 7 : "Jul", 8 : "Aug",
            9 : "Sep", 10 : "Oct", 11 : "Nov", 12 : "Dec", }
MonKeys = sorted( MonDict.keys() )

## Custom Functions

In [8]:
def probDistLLogis( paramDict, npArray ):
    """Uses generalized logistic probability distribution to estimate cumulative
    probilities for each value in the Numpy array, npArray.
    
    Args:
        paramDict (dict): dictionary with best-fit parameter values for a 
                log-logisitic distribution. Must have keys: "k", "scale",
                "loc" which are the 3 required parameters
        npArray (np.ndarray): array from time series of monthly, rolling
                average values
    
    Returns:
        retArray (np.ndarray): cumulative probabilies for each npArray value
    """
    shape = paramDict["k"]
    location = paramDict["loc"]
    scale = paramDict["scale"]
    if shape == 0.0:
        # this is the special case of a logistic distribution with 2 params
        y = ( npArray - location ) / scale
    else:
        # this is the general case of the log-logistic distribution
        takeLogArray = 1.0 - ( shape * ( npArray - location ) / scale )
        useLogArray = np.where( takeLogArray <= 0.0, 1e-7, takeLogArray )
        y = ( -1.0 * ( 1.0 / shape ) ) * np.log( useLogArray )
    # end if
    retArray = 1.0 / ( 1.0 + np.exp( -1.0 * y  ) )
    # return
    return retArray

In [9]:
def ppfLLogis( paramDict, npArray ):
    """Uses generalized logistic probability distribution to estimate cumulative
    probilities for each value in the Numpy array, npArray.
    
    Args:
        paramDict (dict): dictionary with best-fit parameter values for a 
                log-logisitic distribution. Must have keys: "k", "scale",
                "loc" which are the 3 required parameters
        npArray (np.ndarray): array from time series of cumulative probability
                values. All values must be greater than zero and less than or equal to one.
    
    Returns:
        retArray (np.ndarray): cumulative deficit values corresponding to cumulative
                               probabilities
    """
    shape = paramDict["k"]
    location = paramDict["loc"]
    scale = paramDict["scale"]
    if shape == 0.0:
        # this is the special case of a logistic distribution with 2 params
        retArray = location - ( scale *  np.log( ( 1.0 - npArray ) / npArray ) )
    else:
        # this is the general case of the log-logistic distribution
        retArray = location + ( scale * ( ( 1.0 - np.power( ( ( 1.0 - npArray ) / npArray ), shape ) ) / shape ) )
    # end if
    # return
    return retArray

## Test

In [10]:
bas = BAS_KEYS[0]
bas

'Blanco'

In [11]:
InFiler = os.path.normpath( os.path.join( IN_DIR1, bas, "%s_SPEI.xlsx" % bas ) )

In [12]:
llDistParamsDF = pd.read_excel( InFiler, sheet_name="Stats_3mo", header=0, index_col=0 )

In [13]:
display( HTML( llDistParamsDF.head().to_html() ) )

,SPEI Fit Stats
DateTime,
shape_1,-0.081375
scale_1,71.449400
loc_1,-0.420170
shape_2,-0.173454
scale_2,68.012194


In [14]:
display( HTML( llDistParamsDF.tail().to_html() ) )

,SPEI Fit Stats
DateTime,
scale_11,90.023368
loc_11,-84.404699
shape_12,-0.106177
scale_12,97.857679
loc_12,-2.890737


In [15]:
speiResultsDF = pd.read_excel( InFiler, sheet_name="SPEI_3mo", header=0, index_col=0, parse_dates=True )

In [16]:
display( HTML( speiResultsDF.head().to_html() ) )

,CumDef,CumProb,SPEI,5xCumProb
DateTime,,,,
1981-01-01,-30.056408,0.396051,-0.263583,1.980253
1981-02-01,-67.786949,0.351531,-0.381191,1.757653
1981-03-01,-47.938759,0.551958,0.130610,2.759791
1981-04-01,-115.340317,0.580186,0.202371,2.900932
1981-05-01,-145.696564,0.588437,0.223526,2.942184


In [17]:
display( HTML( speiResultsDF.tail().to_html() ) )

,CumDef,CumProb,SPEI,5xCumProb
DateTime,,,,
2022-08-01,-364.142273,0.470958,-0.072862,2.354789
2022-09-01,-286.355835,0.685661,0.483587,3.428303
2022-10-01,-178.918564,0.624979,0.318583,3.124894
2022-11-01,-201.438919,0.184381,-0.898796,0.921903
2022-12-01,-130.513306,0.197219,-0.851598,0.986093


In [18]:
speiResultsDF.at[ExtractTS, "CumProb"], speiResultsDF.at[ExtractTS, "SPEI"], speiResultsDF.at[ExtractTS, "5xCumProb"]

(0.03496904671192169, -1.812311410903931, 0.1748452335596085)

In [19]:
exCumDef = speiResultsDF.at[ExtractTS, "CumDef"]
exCumProb = speiResultsDF.at[ExtractTS, "CumProb"]
ex5xCP = speiResultsDF.at[ExtractTS, "5xCumProb"]
exCumDef, exCumProb, ex5xCP

(-534.3901977539062, 0.03496904671192169, 0.1748452335596085)

In [20]:
mon = 7
cParam = llDistParamsDF.at["shape_%d" % mon, "SPEI Fit Stats" ]
cScale = llDistParamsDF.at["scale_%d" % mon, "SPEI Fit Stats" ]
cLoc = llDistParamsDF.at["loc_%d" % mon, "SPEI Fit Stats" ]
cParam, cScale, cLoc

(-0.1302565917240379, 100.42793207277, -263.8543363491371)

In [21]:
paramDict7 = { "k" : cParam, "loc" : cLoc, "scale" : cScale }
paramDict7

{'k': -0.1302565917240379, 'loc': -263.8543363491371, 'scale': 100.42793207277}

In [22]:
probDistLLogis( paramDict7, np.array( [exCumDef], dtype=np.float32 ) )

array([0.03496906], dtype=float32)

In [23]:
ppfLLogis( paramDict7, np.array( [exCumProb], dtype=np.float32 ) )

array([-534.3902], dtype=float32)

In [24]:
mon = MonKeys[0]
mon, MonDict[mon]

(1, 'Jan')

In [25]:
cParam = llDistParamsDF.at["shape_%d" % mon, "SPEI Fit Stats" ]
cScale = llDistParamsDF.at["scale_%d" % mon, "SPEI Fit Stats" ]
cLoc = llDistParamsDF.at["loc_%d" % mon, "SPEI Fit Stats" ]
cParam, cScale, cLoc

(-0.0813747739699357, 71.44939958636274, -0.4201702207401237)

In [26]:
paramDict1 = { "k" : cParam, "loc" : cLoc, "scale" : cScale }
paramDict1

{'k': -0.0813747739699357,
 'loc': -0.4201702207401237,
 'scale': 71.44939958636274}

In [27]:
tMonCumDef = ppfLLogis( paramDict1, np.array( [exCumProb], dtype=np.float32 ) ) 
tMonCumDef, exCumProb

(array([-208.16429], dtype=float32), 0.03496904671192169)

## All Basins Collate

Loop through and process all basins and months

In [28]:
for bas in BAS_KEYS:
    # get the month to use to set all months and associated information
    InFiler = os.path.normpath( os.path.join( IN_DIR1, bas, "%s_SPEI.xlsx" % bas ) )
    llDistParamsDF = pd.read_excel( InFiler, sheet_name="Stats_3mo", header=0, index_col=0 )
    speiResultsDF = pd.read_excel( InFiler, sheet_name="SPEI_3mo", header=0, index_col=0, parse_dates=True )
    exCumDef = speiResultsDF.at[ExtractTS, "CumDef"]
    exCumProb = speiResultsDF.at[ExtractTS, "CumProb"]
    ex5xCP = speiResultsDF.at[ExtractTS, "5xCumProb"]
    exMon = ExtractTS.month
    # initialize the tracking lists
    exCumProbList = list()
    exCumDefList = list()
    new5xCumProbList = list()
    newCumDefList = list()
    for mon in MonKeys:
        cParam = llDistParamsDF.at["shape_%d" % mon, "SPEI Fit Stats" ]
        cScale = llDistParamsDF.at["scale_%d" % mon, "SPEI Fit Stats" ]
        cLoc = llDistParamsDF.at["loc_%d" % mon, "SPEI Fit Stats" ]
        paramDict = { "k" : cParam, "loc" : cLoc, "scale" : cScale }
        tMonCumDef = ppfLLogis( paramDict, np.array( [exCumProb], dtype=np.float32 ) )
        svCumDef = float( tMonCumDef[0] )
        exCumProbList.append( exCumProb )
        exCumDefList.append( svCumDef )
        new5xCumProbList.append( ex5xCP )
        newCumDefList.append( svCumDef )
        if mon == 7:
            print("Original cumulative deficit %7.2f and new cumulative deficit %7.2f" %
                  (exCumDef, svCumDef) )
        # endif
    # end for
    DataDict = { "2022 Cum Prob" : np.array( exCumProbList, dtype=np.float32 ),
                 "2022 Cum Def" : np.array( exCumDefList, dtype=np.float32 ),
                 "5X Cum Prob" : np.array( new5xCumProbList, dtype=np.float32 ),
                 "Cum Def for 5X prob" : np.array( newCumDefList, dtype=np.float32 ), }
    curDF = pd.DataFrame( index=MonKeys, data=DataDict )
    outXLSX = os.path.normpath( os.path.join( OUT_DIR, bas, "%s_drought_targets.xlsx" % bas ) )
    writer = pd.ExcelWriter( outXLSX )
    workbook  = writer.book
    format1 = workbook.add_format({'num_format': '#,##0.000'})
    cLabel = "Targets"
    curDF.to_excel( writer, sheet_name=cLabel, index_label="Month" )
    # adjust columns
    writer.sheets[cLabel].set_column( 0, 0, 10 )
    for column in curDF:
        column_width = max(curDF[column].astype(str).map(len).max()+6, len(column)+6)
        col_idx = curDF.columns.get_loc(column)
        writer.sheets[cLabel].set_column(col_idx+1, col_idx+1, column_width, format1)
    # end for
    writer.close()
# end of basin for

Original cumulative deficit -534.39 and new cumulative deficit -534.39
Original cumulative deficit -533.06 and new cumulative deficit -533.06
Original cumulative deficit -539.55 and new cumulative deficit -539.55
Original cumulative deficit -528.91 and new cumulative deficit -528.91
Original cumulative deficit -549.22 and new cumulative deficit -549.22
Original cumulative deficit -542.80 and new cumulative deficit -542.80
Original cumulative deficit -538.58 and new cumulative deficit -538.58
Original cumulative deficit -543.72 and new cumulative deficit -543.72
Original cumulative deficit -531.50 and new cumulative deficit -531.50
